In [1]:
!pip install mynn
!pip install cogworks-data
!pip install gensim
!pip mygrad as mg
!pip install matplotlib

ERROR: unknown command "mygrad"


In [4]:
from cogworks_data.language import get_data_path

from pathlib import Path
import json

from gensim.models import KeyedVectors

import pickle
import re
import string

from mynn.layers.dense import dense
from mynn.initializers import glorot_normal
import mygrad as mg
from mynn.optimizers.sgd import SGD

import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
from PIL import Image
import requests
import io

In [3]:
class COCO:
    def __init__(self):

        image_ids = list()
        caption_ids = list()
        image_to_captions = {}
        caption_to_image = {}
        caption_to_text = {}

        self.image_ids = image_ids
        self.caption_ids = caption_ids
        self.image_to_captions = image_to_captions
        self.caption_to_image = caption_to_image
        self.caption_to_text = caption_to_text

        filename = get_data_path("captions_train2014.json")
        with Path(filename).open() as f:
            coco_data = json.load(f)

        self.coco_data = coco_data

        
        with Path(get_data_path('resnet18_features.pkl')).open('rb') as f:
            self.resnet18_features = pickle.load(f)

        for image_info in coco_data['images']:
            if image_info["id"] in self.resnet18_features:
                image_ids.append(image_info['id'])
            
        for caption_info in coco_data['annotations']:
            if caption_info['image_id'] in self.resnet18_features:
                caption_id = caption_info['id']
                image_id = caption_info['image_id']

                caption_ids.append(caption_id)
                
                if image_id not in image_to_captions:
                    image_to_captions[image_id] = []
                image_to_captions[image_id].append(caption_id)

                caption_to_image[caption_id] = image_id

                caption_to_text[caption_id] = caption_info['caption']

    def get_image_to_caption(self, image_id):
        return self.image_to_captions[image_id]

    def get_caption_to_image(self, caption_id):
        return self.caption_to_image[caption_id]
    
    def get_caption_to_text(self, caption_id):
        return self.caption_to_text[caption_id]

    def get_resnet_features(self, image_id):
        return self.resnet18_features[image_id]

    def get_all_captions(self):
        return list(self.caption_to_text.values())

In [32]:
with Path(get_data_path('resnet18_features.pkl')).open('rb') as f:
    resnet18_features = pickle.load(f)

In [5]:
dataset = COCO()
print(len(dataset.image_ids))
print(len(dataset.caption_ids))
print(dataset.coco_data["annotations"][6]["caption"])

82612
413258
A bathroom with a border of butterflies and blue paint on the walls above it.


In [6]:
from gensim.models import KeyedVectors
filename = "glove.6B.200d.txt.w2v"

# this takes a while to load -- keep this in mind when designing your capstone project
glove = KeyedVectors.load_word2vec_format(get_data_path(filename), binary=False)

In [8]:
import re, string
punc_regex = re.compile('[{}]'.format(re.escape(string.punctuation)))
def strip_punc(corpus):
    """ Removes all punctuation from a string.
        
    Parameters
    ----------
    corpus : str

    Returns
    -------
    str
        the corpus with all punctuation removed"""
    # substitute all punctuation marks with ""
    return punc_regex.sub('', corpus)

In [10]:
def to_counter(doc):
    """ 
    Produce word-count of document, removing all punctuation
    and making all the characters lower-cased.
    
    Parameters
    ----------
    doc : str
    
    Returns
    -------
    collections.Counter
        lower-cased word -> count
    
    Examples
    --------
    >>> to_counter("I am a dog.")
    Counter({'a': 1, 'am': 1, 'dog': 1, 'i': 1})
    """
    return Counter(strip_punc(doc).lower().split())

In [11]:
def to_vocab(counter):
    return list(counter.keys())

In [12]:
def to_idf(vocab, counters):
    """ 
    Given the vocabulary, and the word-counts for each document, computes
    the inverse document frequency (IDF) for each term in the vocabulary.
    
    Parameters
    ----------
    vocab : Sequence[str]
        Ordered list of words that we care about.

    counters : Iterable[collections.Counter]
        The word -> count mapping for each document.
    
    Returns
    -------
    numpy.ndarray
        An array whose entries correspond to those in `vocab`, storing
        the IDF for each term `t`: 
                           log10(N / nt)
        Where `N` is the number of documents, and `nt` is the number of 
        documents in which the term `t` occurs.
    """
    N = len(counters)
    nt = [sum(1 if t in counter else 0 for counter in counters) for t in vocab]
    nt = np.array(nt, dtype=float)
    idf_values = np.log10(N / nt)

    return {word: idf_value for word, idf_value in zip(vocab, idf_values)}

In [13]:
all_captions = ""

for caption in dataset.get_all_captions():
    all_captions += " " + caption

word_counter = to_counter(all_captions)
IDF = to_idf(to_vocab(word_counter), word_counter)
print(IDF)

{'a': 0.33185116672607734, 'very': 3.1149268730679474, 'clean': 3.2175892149650953, 'and': 1.8104817000521536, 'well': 2.791620482692814, 'decorated': 3.490590487028833, 'empty': 3.6947104696847575, 'bathroom': 3.615529223637133, 'panoramic': 4.393680474020776, 'view': 2.9465224426785572, 'of': 2.2005558756663146, 'kitchen': 3.490590487028833, 'all': 1.8844779516896735, 'its': 2.645492447014576, 'appliances': 4.393680474020776, 'blue': 2.8622015569785213, 'white': 2.76021201844119, 'with': 2.93128247612182, 'butterfly': 4.092650478356795, 'themed': 3.2175892149650953, 'wall': 2.916559219301114, 'tiles': 4.092650478356795, 'photo': 3.0134692323091703, 'dining': 3.615529223637133, 'room': 2.580767117377921, 'graffitied': 4.393680474020776, 'stop': 3.05125779319857, 'sign': 2.7404679602454327, 'across': 3.916559219301114, 'the': 2.0926504783567954, 'street': 2.9787071260499585, 'from': 4.092650478356795, 'red': 1.8243065644057306, 'car': 2.17619652980687, 'vandalized': 4.393680474020776, 

In [46]:
nt = [sum(1 if t in counter else 0 for counter in word_counter) for t in vocab]
nt = np.array(nt, dtype=float)
print(nt)

NameError: name 'vocab' is not defined

In [14]:
def compute_caption_embedding(caption:str): 
    embedding = 0.0
    new_caption = strip_punc(caption).lower().split()
    for word in new_caption:
        if word not in glove or word not in IDF:
            embedding += np.zeros(glove.vector_size)
        else:
            embedding += glove[word] * IDF[word]

    return embedding / np.linalg.norm(embedding)

In [15]:
first_caption = dataset.caption_to_text[794853]
print(first_caption)

compute_caption_embedding(first_caption)

A table is adorned with wooden chairs with blue accents.


array([ 0.07403626, -0.08645444, -0.01461355, -0.08058415,  0.03162189,
        0.04649204,  0.01198924,  0.0144426 , -0.09208117, -0.07249843,
        0.02825571,  0.05389195,  0.04787353, -0.05595864,  0.09812161,
       -0.02290024, -0.00197943, -0.00884037,  0.07112002,  0.03627918,
        0.12910913,  0.4115067 ,  0.09203697, -0.08070949,  0.03344185,
       -0.08311874, -0.02228023, -0.06891457, -0.03464694, -0.01177409,
        0.0871267 , -0.04115307,  0.03862696,  0.06333715, -0.07740407,
       -0.04316813, -0.07821321, -0.06989071,  0.06622027, -0.00984694,
        0.02702883,  0.05727576,  0.00965605, -0.03103598, -0.03518967,
        0.02246292,  0.19189441, -0.03683182, -0.03208426,  0.08567183,
        0.00465727,  0.0690013 ,  0.00651788,  0.09942149,  0.00732088,
        0.02781615, -0.0143294 , -0.00172403,  0.08321367,  0.01142737,
        0.0659427 ,  0.0399661 , -0.05419531,  0.04281747, -0.06797048,
       -0.01042461, -0.12848227, -0.0648772 ,  0.07523556, -0.02

In [19]:
class EmbeddingModel:
    def __init__(self):
        self.linear1 = dense(512, 200, weight_initializer=glorot_normal)
    def __call__(self, x):
        output = self.linear1(x)
        return output / mg.linalg.norm(output)
    @property
    def parameters(self):
        return self.linear1.parameters

In [20]:
def get_triplet_batch(coco_data_organizer):
    triplet_batch = []
    num_captions = len(coco_data_organizer.caption_ids)

    for i in range(num_captions):
        caption_id = coco_data_organizer.caption_ids[i]
        true_image_id = coco_data_organizer.get_caption_to_image(caption_id)

        if i < num_captions - 1:
            confusor_caption_id = coco_data_organizer.caption_ids[i + 1]
            confusor_image_id = coco_data_organizer.get_caption_to_image(confusor_caption_id)
        else:
            confusor_caption_id = coco_data_organizer.caption_ids[0]
            confusor_image_id = coco_data_organizer.get_caption_to_image(confusor_caption_id)

        triplet_batch.append([caption_id, true_image_id, confusor_image_id])

    return triplet_batch

In [23]:
triplet_batch = get_triplet_batch(dataset)
counter = 0

for caption_id, true_image_id, confuser_image_id in triplet_batch:
    if true_image_id == confuser_image_id:
        counter += 1

print(counter)

3397


In [25]:
def custom_loss(true_embedding, confusor_embedding, caption_embedding):
    # print(true_embedding.shape, confusor_embedding.shape, caption_embedding.shape)
    sim_true = mg.einsum("nd,nd -> n", true_embedding, caption_embedding)
    sim_confusor = mg.einsum("nd,nd -> n", confusor_embedding, caption_embedding)
    return mg.nnet.losses.margin_ranking_loss(sim_true, sim_confusor, 1, margin=0.25)

In [28]:
caption_embeddings = {}
for caption_id in dataset.caption_ids:
    cap_text = dataset.get_caption_to_text(caption_id)
    caption_embeddings[caption_id] = compute_caption_embedding(cap_text)

In [29]:
coco = COCO()
batch_size = 32
model = EmbeddingModel()
lr = 1e-3
momentum = 0.9
optimizer = SGD(model.parameters, learning_rate=lr, momentum = momentum)

In [30]:
triplet_data = get_triplet_batch(coco)
num_triplets = len(triplet_data)
train_size = 500

train_data = np.array(triplet_data[:train_size])
test_data = triplet_data[train_size:]

train_indices = list(range(train_size))

In [31]:
test = model.parameters 

for epoch in range(10):
    total_loss = 0.0
    for i in range(0, train_size // batch_size):
        batch_idxs = train_indices[i * batch_size : (i + 1) * batch_size]

        caption_ids = train_data[batch_idxs][:, 0]
        true_image_ids = train_data[batch_idxs][:, 1]
        confusor_image_ids = train_data[batch_idxs][:, 2]

        true_caption_embeddings = [caption_embeddings[caption_id] for caption_id in caption_ids]
        true_caption_embeddings = np.array(true_caption_embeddings)

        true_images_desc = [coco.get_resnet_features(image_id) for image_id in true_image_ids]
        true_images_desc = np.vstack(true_images_desc)

        confusor_image_desc = [coco.get_resnet_features(image_id) for image_id in confusor_image_ids]
        confusor_image_desc = np.vstack(confusor_image_desc)

        true_embeddings = model(true_images_desc)
        confusor_embeddings = model(confusor_image_desc)

        # NEED TO RETURN THIS TO MG.EINSUM
        sim_true = mg.einsum("nd,nd -> n", true_embeddings, true_caption_embeddings)
        sim_confusor = mg.einsum("nd,nd -> n", confusor_embeddings, true_caption_embeddings)

        loss = mg.nnet.losses.margin_ranking_loss(sim_true, sim_confusor, 1, margin=0.25)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        print(loss.item())
        print(model.parameters == test)
        test = model.parameters
        #print(loss.item())
    avg_loss = total_loss / (train_size // batch_size)
    print(f"Epoch {epoch + 1}, Loss: {avg_loss:.4f}")
    break

    

0.24993935227394104
True
0.2501078248023987
True
0.2506135404109955
True
0.2497905194759369
True
0.25102102756500244
True
0.250204861164093
True
0.25134217739105225
True
0.25132524967193604
True
0.24873808026313782
True
0.24963775277137756
True
0.2491825968027115
True
0.250201940536499
True
0.2500125467777252
True
0.25143277645111084
True
0.2502344846725464
True
Epoch 1, Loss: 0.2503
